In [ ]:
# --- LAMMPS MD SIMULATION SCEHDULER ---
# 1. Specify melt/quench parameters
# 2. Specify the structure from Random_Carbon_Structures
# This script will run 5 MD simulations on the given structure with the given parameters
# Run this cell for each structure you want to simulate before running the next cell

import subprocess, sys, os


# --- SIMULATION PARAMETERS --- 
structure = "Random_Carbon_Structures/200atoms_3.5gcm_1.76minsep.data"   # Relative filepath to the structure

melt_timesteps = 5000 # in femtoseconds
t_melt = 9000
cool_timesteps = 10000
t_cool = 300

repeats = 10
#------------------------------

def submit_lammps(structure, melt_timesteps, t_melt, cool_timesteps, t_cool, run): 
    
    # Job submission using "qsub"
    structure = os.path.abspath(structure)
    simulation_variables = (
        f"STRUCTURE={structure},"
        f"MELT_TIMESTEPS={melt_timesteps},"
        f"T_MELT={t_melt},"
        f"COOL_TIMESTEPS={cool_timesteps},"
        f"T_COOL={t_cool}"
    )
    cmd = ["qsub", "-v", simulation_variables, "lammps_scheduler.sh", str(run)]
    submission = subprocess.run(cmd, capture_output=True, text=True)

    # Job submission failure checks
    if submission.returncode != 0:
        print("qsub failed:", submission.stderr or submission.stdout, file=sys.stderr)
        sys.exit(1)
    
    # Parsing job_ID
    out = submission.stdout.strip()
    print(out)
    return None

# Runs 5 MD simulations, named 1-5
if repeats < 15 and (melt_timesteps+cool_timesteps) < 100000: # Fat-finger checker
    for run in range (1, repeats+1):
        submit_lammps(structure, melt_timesteps, t_melt, cool_timesteps, t_cool, run)
else:
    print("ERROR:Fat-finger prevention; reduce computational cost or loosen restrictions")

In [ ]:
# ------ FILE CLEANUP ------ 
# 1. Waits until all simulations are done
# 2. Moves all MD files to LAMMPS_simulations/ 
# 3. Groups all repeat runs into subdirectories 


# ------ WAIT FUNCTION ------
import subprocess
import time

WAIT_TIME = 600  # seconds
USER = "scat9451"

while True:
    job_status = subprocess.run(["qstat", "-u", USER], capture_output=True, text=True)
    if not job_status.stdout.strip():  # empty means no jobs
        break
    print("Active jobs found - waiting ...")
    time.sleep(WAIT_TIME)

print("No active jobs found - proceeding ...")
# -----------------------------

# ------ FILE ORGANISER ------
import os, re, shutil

cwd = os.getcwd()
LAMMPS_dir = os.path.join(cwd, "LAMMPS_simulations")
os.makedirs(LAMMPS_dir, exist_ok=True)

LAMMPS_regex_pattern = re.compile(r'^run_(\d+)_(C_nvt_.+)$')

# Moves all run files into LAMMPS_simulations
for directory in os.listdir(cwd):
    match = LAMMPS_regex_pattern.fullmatch(directory)
    if match:
        run_files = os.path.join(cwd, directory)
        destination = os.path.join(LAMMPS_dir, directory)
        if os.path.exists(destination): continue
        shutil.move(run_files, destination)

# Groups all repeat runs into subdirectories 
for directory in os.listdir(LAMMPS_dir):
    match = LAMMPS_regex_pattern.fullmatch(directory)
    if match:
        run_files = os.path.join(LAMMPS_dir, directory)
        simulation_parameters = match.group(2)
        simulation_dir = os.path.join(LAMMPS_dir, f"run_{simulation_parameters}")
        os.makedirs(simulation_dir, exist_ok=True)
        if os.path.exists(os.path.join(simulation_dir, directory)):continue
        shutil.move(run_files, simulation_dir)

# Deletes empty log files 
log_file_regex = re.compile(r'.*\.log$')
cwd = os.getcwd()
files = os.listdir(cwd)
for file in files:
    if (log_file_regex.fullmatch(file)
        and os.path.getsize(file) == 0):
        os.remove(os.path.join(cwd, file))